# Finite-State Machine
The pattern finite-state machine appears in our analytical model as well as in programming. We need to look into this pattern in detail.

[Finite-state machines](https://en.wikipedia.org/wiki/Finite-state_machine) can be subdivided into acceptors, classifiers, transducers and sequencers. 

*Acceptors* (also called detectors or recognizers) produce binary output, indicating whether or not the received input is accepted. Each state of an acceptor is either accepting or non accepting. Once all input has been received, if the current state is an accepting state, the input is accepted; otherwise it is rejected. 

## `Control.Monad.State`
The haskell package `Control.Monad.State` has a main context called `MonadState`:
```haskell
class Monad m => MonadState s m | m -> s where
    get :: m s
    put :: s -> m ()
    state :: (s -> (a, s)) -> m a
```

and few utilities around it:

```haskell
modify :: MonadState s m => (s -> s) -> m ()
gets :: MonadState s m => (s -> a) -> m a
```

The $s$ type variable represents a type of state. It is uniquely determined by the monad
$m$ as specified by the $| m -> s$ part. For example State Integer is a concrete monad
with a state of type Integer. The `get` method resembles ask from Reader—it gives
access to the current state. The `gets` function is the counterpart to asks. The `put`
method is analogous to tell—it sets the current state. The `state` method combines
both reader and writer functionalities. Let’s compare these three functions as follows:

```haskell
reader :: MonadReader r m => (r -> a) -> m a
writer :: MonadWriter w m => (a, w) -> m a
state :: MonadState s m => (s -> (a, s)) -> m a
```
All of them construct a computation in the corresponding Monad from its internal representation (a function, a pair, and a function returning a pair, respectively).


## Some examples
### A simple game
This example is from [State Monad](https://wiki.haskell.org/State_Monad).

In [2]:
import Control.Monad.State

In [7]:
type GameValue = Int
type GameState = (Bool, Int)

playGame :: String -> State GameState GameValue
playGame []     = do
    (_, score) <- get
    return score

playGame (x:xs) = do
    (on, score) <- get
    case x of
         'a' | on -> put (on, score + 1)
         'b' | on -> put (on, score - 1)
         'c'      -> put (not on, score)
         _        -> put (on, score)
    playGame xs

startState = (False, 0)

In [8]:
print $ evalState (playGame "abcaaacbbcabbab") startState

2

### A coin example
This material is from [Using the Haskell State Monad](https://user.it.uu.se/~justin/Hugo/post/haskel_state_random/).

In [5]:
import System.Random
import Control.Monad
import Control.Monad.State

randomSt :: (RandomGen g, Random  a) => State g a
randomSt = state random 

threeCoinsState :: State StdGen (Bool, Bool , Bool)
threeCoinsState = do
  a <- randomSt 
  b <- randomSt
  c <- randomSt
  return (a,b,c)


Most monads do not do anything on their own, they are just sequences of actions. You need to do something to make those actions happen. The required magic for the state monad is `runState`.

In [6]:
runState threeCoinsState (mkStdGen 21)

((True,False,False),StdGen {unStdGen = SMGen 5638754522534429640 489215147674969543})

#### The two entries from `runState`

The first entry is the result from `runState`, one can use `evalState` to get it.

In [7]:
evalState threeCoinsState (mkStdGen 21)

(True,False,False)

mkStdGen 210

In [8]:
r=mkStdGen 21

In [9]:
(x1,r1)=next r

In [10]:
print x1

-5847039462251234981

In [11]:
print r1

StdGen {unStdGen = SMGen 4660324227184490554 489215147674969543}

In [12]:
import System.Directory
getAppUserDataDirectory "cabal"

"/home/zzhang/.cabal"